This notebook runs a `Qallse` model in simulation mode and shows how to simply visualize the results using utilities defined in module `hepqpr.qallse.plotting`.

# Import and setup

In [1]:
# import modules
import pandas as pd
import numpy as np

from hepqpr.qallse.plotting import *
from hepqpr.qallse import *
from hepqpr.qallse.dsmaker import create_dataset

# initialise the plotting module in "notebook" mode
set_notebook_mode()

# initialise the logging module
import logging
logging.basicConfig()
fmt = logging.Formatter("%(asctime)s.%(msecs)03d %(levelname)s %(name)s: %(message)s", datefmt='%H:%M:%S')
for handler in logging.getLogger().handlers: handler.setFormatter(fmt)
logging.getLogger('hepqpr').setLevel(logging.DEBUG)

0
[(-4, 4.4), (4, inf), (-inf, -4)]
0
[(0.0, 0.8)]
1
[(0.0, 0.8), (0.5, 1.3)]
2
[(0.0, 0.8), (0.5, 1.3), (1.0, 1.8)]
3
[(0.0, 0.8), (0.5, 1.3), (1.0, 1.8), (1.5, 0.2999999999999998)]


# Define run config

In [2]:
# == DATASET CONFIG

dsmaker_config = dict(
    density=.02, # 1% 
)

# == INPUT CONFIG

# whether or not to add missing doublets to the input
add_missing = True 

# == RUN CONFIG

model_class = Qallse # model class to use
extra_config = dict() # configuration arguments overriding the defaults

# Generate the data

In [3]:
import tempfile

tempdir = tempfile.TemporaryDirectory()
print(f'using {tempdir.name}')
metas, path = create_dataset(output_path=tempdir.name, gen_doublets=True, random_seed=240834366, **dsmaker_config)

using /var/folders/7r/y3xzyywn4mg2tfhg0sgjqx_w0000gn/T/tmp0jf2mwf6


20:02:54.900 DEBUG hepqpr.qallse.dsmaker.dsmaker: Loaded 120939 hits from /Users/parkerreid/hepqpr-qallse/src/hepqpr/qallse/dsmaker/data/event000001000.
20:02:54.928 DEBUG hepqpr.qallse.dsmaker.dsmaker: Filtered hits from barrel. Remaining hits: 65518.
20:02:54.983 DEBUG hepqpr.qallse.dsmaker.dsmaker: Dropped double hits. Remaining hits: 55446.
20:02:55.019 DEBUG hepqpr.qallse.dsmaker.dsmaker: High Pt hits: 32302/1152
20:02:55.078 DEBUG hepqpr.qallse.dsmaker.dsmaker:   num_hits=1152
20:02:55.079 DEBUG hepqpr.qallse.dsmaker.dsmaker:   num_tracks=163
20:02:55.080 DEBUG hepqpr.qallse.dsmaker.dsmaker:   num_important_tracks=17
20:02:55.080 DEBUG hepqpr.qallse.dsmaker.dsmaker:   num_noise=260
20:02:55.081 DEBUG hepqpr.qallse.dsmaker.dsmaker:   random_seed=240834366
20:02:55.081 DEBUG hepqpr.qallse.dsmaker.dsmaker:   time=2019-12-31T20:02:55.078761
20:02:55.566 INFO hepqpr.qallse.dsmaker.dsmaker: Doublets (len=5626) generated in f/var/folders/7r/y3xzyywn4mg2tfhg0sgjqx_w0000gn/T/tmp0jf2mwf6/e

In [4]:
with open(path + '-meta.json') as f:
    print(f.read())

{
    "num_hits": 1152,
    "num_tracks": 163,
    "num_important_tracks": 17,
    "num_noise": 260,
    "random_seed": 240834366,
    "time": "2019-12-31T20:02:55.078761",
    "params": {
        "input_path": "/Users/parkerreid/hepqpr-qallse/src/hepqpr/qallse/dsmaker/data/event000001000",
        "output_path": "/var/folders/7r/y3xzyywn4mg2tfhg0sgjqx_w0000gn/T/tmp0jf2mwf6",
        "density": 0.02,
        "high_pt_cut": 1.0,
        "double_hits_ok": false,
        "gen_doublets": true,
        "prefix": null,
        "random_seed": 240834366,
        "phi_bounds": null,
        "min_hits_per_track": 5
    }
}


# Execute the model

## Load the data

In [5]:
# load data
dw = DataWrapper.from_path(path)
doublets = pd.read_csv(path + '-doublets.csv')
if add_missing:
    doublets = dw.add_missing_doublets(doublets)
else:
    p, r, ms = dw.compute_score(doublets)
    print(f'got {len(doublets)}.')
    print(f'  Input precision (%): {p*100:.4f}, recall (%): {r*100:.4f}, missing: {len(ms)}')

got 5626 doublets.
  Input precision (%): 2.2939, recall (%): 98.3333
    New precision (%): 2.3319


## Build the model

In [6]:
%%time

# instantiate qallse
model = model_class(dw, **extra_config)
# build the qubo
model.build_model(doublets=doublets)
sliceContainer = model.to_qubo()

20:02:55.780 DEBUG hepqpr.qallse.qallse: using config:
20:02:55.781 DEBUG hepqpr.qallse.qallse:     cheat: False
20:02:55.782 DEBUG hepqpr.qallse.qallse:     max_layer_span: 2
20:02:55.782 DEBUG hepqpr.qallse.qallse:     num_multiplier: -1
20:02:55.783 DEBUG hepqpr.qallse.qallse:     qplet_max_dcurv: 0.0001
20:02:55.783 DEBUG hepqpr.qallse.qallse:     qplet_max_strength: -0.2
20:02:55.784 DEBUG hepqpr.qallse.qallse:     qubo_bias_weight: 0
20:02:55.784 DEBUG hepqpr.qallse.qallse:     qubo_conflict_strength: 1
20:02:55.785 DEBUG hepqpr.qallse.qallse:     rz_power: 1
20:02:55.786 DEBUG hepqpr.qallse.qallse:     strength_bounds: None
20:02:55.787 DEBUG hepqpr.qallse.qallse:     tplet_max_curv: 0.0008
20:02:55.788 DEBUG hepqpr.qallse.qallse:     tplet_max_drz: 0.1
20:02:55.789 DEBUG hepqpr.qallse.qallse:     volayer_power: 2
20:02:55.790 DEBUG hepqpr.qallse.qallse:     xy_power: 1
20:02:55.792 DEBUG hepqpr.qallse.qallse:     xy_relative_strength: 0.5
20:02:56.015 INFO hepqpr.qallse.qallse:

CPU times: user 498 ms, sys: 12.9 ms, total: 511 ms
Wall time: 515 ms


## Sample the QUBO

Here, we use [_qbsolv_](https://github.com/dwavesystems/qbsolv) with default arguments. This means only classical algorithms.

In [7]:
%%time
# execute the qubo
responseContainer = model.sample_qubo_slices(Q=sliceContainer)
#response = model.sample_qubo_slices(Q=qslice)

0   0
{('21471_31460_38560', '21471_31460_38560'): 0, ('21471_38560_45453', '21471_38560_45453'): 0, ('21525_37443_44211', '21525_37443_44211'): 0, ('21525_29691_37443', '21525_29691_37443'): 0, ('21663_30775_37454', '21663_30775_37454'): 0, ('21525_29691_44211', '21525_29691_44211'): 0, ('21663_30775_44225', '21663_30775_44225'): 0, ('45453_76929_84225', '45453_76929_84225'): 0, ('31460_38560_45453', '31460_38560_45453'): 0, ('31460_38560_76929', '31460_38560_76929'): 0, ('31460_45453_76929', '31460_45453_76929'): 0, ('45453_76929_91250', '45453_76929_91250'): 0, ('45453_84225_91250', '45453_84225_91250'): 0, ('33923_40321_72075', '33923_40321_72075'): 0, ('33923_40321_78784', '33923_40321_78784'): 0, ('73401_80516_86969', '73401_80516_86969'): 0, ('33903_71593_78393', '33903_71593_78393'): 0, ('33923_72075_78784', '33923_72075_78784'): 0, ('35134_73401_80516', '35134_73401_80516'): 0, ('35134_41791_73401', '35134_41791_73401'): 0, ('35134_41791_80516', '35134_41791_80516'): 0, ('7564

20:02:56.529 INFO hepqpr.qallse.qallse: QUBO of size 277 sampled in 0.23s (seed 1536043309).


0   1
{('22334_31028_37692', '22334_31028_37692'): 0, ('22334_31028_44398', '22334_31028_44398'): 0, ('22334_37692_44398', '22334_37692_44398'): 0, ('22373_30087_36971', '22373_30087_36971'): 0, ('22373_36971_43305', '22373_36971_43305'): 0, ('22373_30087_43305', '22373_30087_43305'): 0, ('22449_31055_38310', '22449_31055_38310'): 0, ('22527_30159_43917', '22527_30159_43917'): 0, ('22527_30159_37020', '22527_30159_37020'): 0, ('44880_83796_90703', '44880_83796_90703'): 0, ('72845_85723_91559', '72845_85723_91559'): 0, ('32192_39479_77859', '32192_39479_77859'): 0, ('72807_79334_85701', '72807_79334_85701'): 0, ('72845_79384_85723', '72845_79384_85723'): 0, ('33923_40321_72075', '33923_40321_72075'): 0, ('33923_40321_78784', '33923_40321_78784'): 0, ('34934_41176_79384', '34934_41176_79384'): 0, ('74191_80778_87530', '74191_80778_87530'): 0, ('33903_71593_78393', '33903_71593_78393'): 0, ('33992_40085_72140', '33992_40085_72140'): 0, ('33923_72075_78784', '33923_72075_78784'): 0, ('3486

20:02:56.922 INFO hepqpr.qallse.qallse: QUBO of size 678 sampled in 0.62s (seed 1536043309).


0   2
{('22449_31055_38310', '22449_31055_38310'): 0, ('22527_30159_43917', '22527_30159_43917'): 0, ('22527_30159_37020', '22527_30159_37020'): 0, ('22937_31253_37871', '22937_31253_37871'): 0, ('22937_37871_44524', '22937_37871_44524'): 0, ('22937_31253_44524', '22937_31253_44524'): 0, ('23234_30490_44070', '23234_30490_44070'): 0, ('23234_30490_37263', '23234_30490_37263'): 0, ('23248_30508_37276', '23248_30508_37276'): 0, ('23234_37263_44070', '23234_37263_44070'): 0, ('23248_30508_43563', '23248_30508_43563'): 0, ('44583_76015_83183', '44583_76015_83183'): 0, ('44880_83796_90703', '44880_83796_90703'): 0, ('44583_83183_89930', '44583_83183_89930'): 0, ('73363_86293_92029', '73363_86293_92029'): 0, ('72845_85723_91559', '72845_85723_91559'): 0, ('72845_79384_85723', '72845_79384_85723'): 0, ('34934_41176_79384', '34934_41176_79384'): 0, ('74191_80778_87530', '74191_80778_87530'): 0, ('33992_40085_72140', '33992_40085_72140'): 0, ('74191_87530_93370', '74191_87530_93370'): 0, ('3493

20:02:57.354 INFO hepqpr.qallse.qallse: QUBO of size 801 sampled in 1.05s (seed 1536043309).


CPU times: user 1.05 s, sys: 7.49 ms, total: 1.05 s
Wall time: 1.06 s


## Parse the results

In [8]:
# get all output doublets
all_doublets = model.process_sample_slices(responseContainer=responseContainer)
# recreate tracks and resolve remaining conflicts
final_tracks, final_doublets = TrackRecreaterD().process_results(all_doublets)

20:02:57.368 INFO hepqpr.qallse.track_recreater: Found 2 conflicting doublets
20:02:57.369 DEBUG hepqpr.qallse.track_recreater: Conflicts added: [35080, 73363].
20:02:57.370 INFO hepqpr.qallse.track_recreater: Added 1 conflicting doublets


# Evaluate the results

## Print statistics and scores

In [9]:
# stats about the qbsolv run
#en0 = dw.compute_energy(Q)
#en = response.record.energy[0]

#print(f'SAMPLE -- energy: {en:.4f}, ideal: {en0:.4f} (diff: {en-en0:.6f})')
#occs = response.record.num_occurrences
#print(f'          best sample occurrence: {occs[0]}/{occs.sum()}')

# scores
p, r, missings = dw.compute_score(final_doublets)
print(f'SCORE  -- precision (%): {p * 100}, recall (%): {r * 100}, missing: {len(missings)}')
trackml_score = dw.compute_trackml_score(final_tracks)
print(f'          tracks found: {len(final_tracks)}, trackml score (%): {trackml_score * 100}')

SCORE  -- precision (%): 98.96907216494846, recall (%): 80.0, missing: 24
          tracks found: 20, trackml score (%): 78.34065486879372


### Plot the results

You have the following at your disposal:
```python
iplot_results(dw, final_doublets, missings, dims=list('xy'))
iplot_results_tracks(dw, final_doublets, missings, dims=list('zr'))
iplot_results_any(dw, dw.real_doublets, dims=list('xy'))
```

For 3D plots, simply define `dims` as 3-dimensional. For example:
```python
iplot_results(dw, final_doublets, missings, dims=list('zxy'))
```

In [10]:
iplot_results(dw, final_doublets, missings, dims=list('xy'))

In [11]:
iplot_results(dw, final_doublets, missings, dims=list('zr'))

In [12]:
iplot_results(dw, final_doublets, missings, dims=list('zxy'))